# Сесия 5 – Мултиагентен оркестратор

Демонстрира проста двуагентна последователност (Изследовател -> Редактор) с използване на Foundry Local.


### Обяснение: Инсталиране на зависимости
Инсталира `foundry-local-sdk` и `openai`, необходими за достъп до локални модели и завършване на чатове. Идемпотентно.


# Сценарий
Реализира минимален модел на оркестратор с два агента:
- **Агент изследовател** събира кратки фактически точки
- **Агент редактор** пренаписва за яснота на изпълнителско ниво

Демонстрира споделена памет за всеки агент, последователно предаване на междинни резултати и проста функция за обработка. Може да се разшири с повече роли (напр. Критик, Проверител) или паралелни клонове.

**Променливи на средата:**
- `FOUNDRY_LOCAL_ALIAS` - Модел по подразбиране за използване (по подразбиране: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Основен модел за агента (замества ALIAS)
- `AGENT_MODEL_EDITOR` - Модел за агента редактор (по подразбиране основния)

**Референция за SDK:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Как работи:**
1. **FoundryLocalManager** автоматично стартира услугата Foundry Local
2. Изтегля и зарежда посочения модел (или използва кеширана версия)
3. Осигурява съвместим крайна точка с OpenAI за взаимодействие
4. Всеки агент може да използва различен модел за специализирани задачи
5. Вградена логика за повторение обработва временни грешки безпроблемно

**Основни характеристики:**
- ✅ Автоматично откриване и инициализация на услугата
- ✅ Управление на жизнения цикъл на модела (изтегляне, кеширане, зареждане)
- ✅ Съвместимост с OpenAI SDK за познат API
- ✅ Поддръжка на множество модели за специализация на агенти
- ✅ Надеждно обработване на грешки с логика за повторение
- ✅ Локално извеждане (без необходимост от облачен API)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Обяснение: Основни импорти и типизация
Въвежда dataclasses за съхранение на съобщенията на агентите и подсказки за типизация за по-голяма яснота. Импортира Foundry Local manager + OpenAI клиент за последващи действия на агентите.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Обяснение: Инициализация на модела (SDK модел)
Използва Foundry Local Python SDK за надеждно управление на модели:
- **FoundryLocalManager(alias)** - Автоматично стартира услугата и зарежда модела чрез псевдоним
- **get_model_info(alias)** - Превръща псевдонима в конкретен ID на модела
- **manager.endpoint** - Осигурява крайна точка за клиента на OpenAI
- **manager.api_key** - Осигурява API ключ (опционално за локално използване)
- Поддържа отделни модели за различни агенти (основен срещу редактор)
- Вградена логика за повторение с експоненциално увеличаване на интервала за устойчивост
- Проверка на връзката, за да се гарантира, че услугата е готова

**Основен SDK модел:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Управление на жизнения цикъл:**
- Мениджърите се съхраняват глобално за правилно почистване
- Всеки агент може да използва различен модел за специализация
- Автоматично откриване на услуги и управление на връзката
- Плавно повторение с експоненциално увеличаване на интервала при неуспехи

Това гарантира правилна инициализация преди започване на оркестрацията на агентите.

**Референция:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Обяснение: Класове Agent и Memory
Определя лек клас `AgentMsg` за записи в паметта и `Agent`, който обхваща:
- **Роля на системата** - Персона и инструкции на агента
- **История на съобщенията** - Поддържа контекста на разговора
- **Метод act()** - Изпълнява действия с подходящо управление на грешки

Агентът може да използва различни модели (основна срещу редакторска) и поддържа изолиран контекст за всеки агент. Този модел позволява:
- Запазване на паметта между действията
- Гъвкаво задаване на модели за всеки агент
- Изолация на грешки и възстановяване
- Лесно свързване и оркестрация


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Обяснение: Оркестриран конвейер
Създава два специализирани агента:
- **Изследовател**: Използва основния модел, събира фактическа информация
- **Редактор**: Може да използва отделен модел (ако е конфигуриран), усъвършенства и пренаписва

Функцията `pipeline`:
1. Изследователят събира сурова информация
2. Редакторът я усъвършенства в готов за изпълнение резултат
3. Връща както междинните, така и окончателните резултати

Този модел позволява:
- Специализация на моделите (различни модели за различни роли)
- Подобряване на качеството чрез многоетапна обработка
- Проследимост на трансформацията на информацията
- Лесно разширяване с повече агенти или паралелна обработка


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Обяснение: Изпълнение на тръбопровода и резултати
Изпълнява многослоен тръбопровод с агенти върху въпрос, свързан с съответствие и латентност, за да демонстрира:
- Многоетапна трансформация на информацията
- Специализация и сътрудничество между агентите
- Подобряване на качеството на изхода чрез прецизиране
- Проследимост (запазване както на междинните, така и на крайните резултати)

**Структура на резултата:**
- `question` - Оригиналният въпрос на потребителя
- `research` - Суров изход от изследването (фактически точки)
- `final` - Прецизирано резюме за ръководители
- `models` - Кои модели са използвани за всеки етап

**Идеи за разширение:**
1. Добавяне на агент Критик за преглед на качеството
2. Реализиране на паралелни агенти за изследване на различни аспекти
3. Добавяне на агент Проверител за проверка на фактите
4. Използване на различни модели за различни нива на сложност
5. Реализиране на обратни връзки за итеративно подобрение


### Напреднали: Персонализирана конфигурация на агент

Опитайте да персонализирате поведението на агента, като промените променливите на средата преди да изпълните клетката за инициализация:

**Налични модели:**
- Използвайте `foundry model ls` в терминала, за да видите всички налични модели
- Примери: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b и др.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за каквито и да било недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
